In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.callbacks import EarlyStopping
from keras.layers import ConvLSTM2D

adm_name = gpd.read_file('data/TSDM/Mandal_Boundary.shp')
df = pd.read_csv('data/cropfire_taluk.csv', index_col=False)


import time                                      #to calculate time taken to run the model
start_time = time.time()                         #start time of the program

place = []
deviance = []

for i in df.place.unique():
  dataframe = pd.DataFrame()
  dataframe = df.loc[df['place'] == i]
  del dataframe['place']
  dataframe = dataframe.reset_index()
  del dataframe['index']

  #Convert pandas dataframe to numpy array
  dataset = dataframe.fireCount
  dataset = dataset.astype('float32') #COnvert values to float
  #LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
  # normalize the dataset
  scaler = MinMaxScaler(feature_range=(0, 1)) #Also try QuantileTransformer
  dataset = np.array(dataset).reshape(-1,1)
  dataset = scaler.fit_transform(dataset)
  train_size = int(len(dataset)-365)
  test_size = len(dataset) - train_size
  train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
  print(test_size)


  def to_sequences(dataset, seq_size=1):
      x = []
      y = []

      for i in range(len(dataset)-seq_size-1):
          #print(i)
          window = dataset[i:(i+seq_size), 0]
          x.append(window)
          y.append(dataset[i+seq_size, 0])
          
      return np.array(x),np.array(y)
      

  seq_size = 10  # Number of time steps to look back 
  #Larger sequences (look further back) may improve forecasting.

  trainX, trainY = to_sequences(train, seq_size)
  testX, testY = to_sequences(test, seq_size)

  print("Shape of training set: {}".format(trainX.shape))
  print("Shape of test set: {}".format(testX.shape))

  trainX = trainX.reshape((trainX.shape[0], 1, 1, 1, seq_size))
  testX = testX.reshape((testX.shape[0], 1, 1, 1, seq_size))

  model = Sequential()
  model.add(ConvLSTM2D(filters=64, kernel_size=(1,1), activation='relu', input_shape=(1, 1, 1, seq_size)))
  model.add(Flatten())
  model.add(Dense(32))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mean_squared_error')
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=20,verbose=1, mode='auto', restore_best_weights=True)
  model.summary()
  print('Train...')

  model.fit(trainX, trainY, validation_data=(testX, testY),
            verbose=2, epochs=100)


  # make predictions

  trainPredict = model.predict(trainX)
  testPredict = model.predict(testX)

  trainPredict = scaler.inverse_transform(trainPredict)
  trainY = scaler.inverse_transform([trainY])
  testPredict = scaler.inverse_transform(testPredict)
  testY = scaler.inverse_transform([testY])

  y_true = scaler.inverse_transform(dataset)[-360:]
  y_pred = testPredict
  y_pred = np.round(y_pred,0)

  if np.max(y_pred) > np.max(y_true):
    deviance.append('Positive')
  else:
    deviance.append('Negative')
  place.append(i)


  plt.plot(y_true, label='Actual')
  plt.plot(y_pred, label='Predicted')
  plt.legend()
  plt.show()

result = pd.DataFrame()
result['Place'], result['Deviance'] = place, deviance

print("--- %s seconds ---" % (time.time() - start_time)) #print total time taken to run code 

result1 = result.loc[result['Deviance'] == 'Positive']

result = result.sort_values(by='Place')

result2 = result1.rename(columns = {'Place': 'N_Revenue'}, inplace = True)
dev = []
ar = []
for i in adm_name.N_Revenue:
  if i in list(result1.N_Revenue):
    dev.append(1)
    ar.append(i)
  else:
    dev.append(0)
    ar.append(i)
    
adm_name = adm_name.sort_values(by='N_Revenue')
adm_name['deviance']=dev

ax = adm_name.plot(column='deviance', legend=True, legend_kwds={'label': "Taluks having Positive Deviance",

                        'orientation': "vertical"}, figsize=(12, 12));

ax.set_title('Telangana Data Powered Positive Deviance for Crop Fire')

fig = ax.get_figure()
fig.savefig("output/LSTM_taluk.png")

In [ ]:
dev = []
ar = []
for i in adm_name.N_Revenue:
  if i in list(result1.N_Revenue):
    dev.append(1)
    ar.append(i)
  else:
    dev.append(0)
    ar.append(i)
    
adm_name = adm_name.sort_values(by='N_Revenue')
adm_name['deviance']=dev

ax = adm_name.plot(column='deviance', legend=True, legend_kwds={'label': "Taluks having Positive Deviance",

                        'orientation': "vertical"}, figsize=(12, 12));

ax.set_title('Telangana Data Powered Positive Deviance for Crop Fire')

fig = ax.get_figure()
fig.savefig("output/LSTM_taluk.png")